# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [19]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [20]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")

API key looks good so far


In [21]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [22]:
# set up environment
openai = OpenAI(api_key=api_key)
# first run "ollama run llama3.2" to start the local server
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="")

In [23]:
sys_prompt_default = "You are a helpful programming assistant."
user_prompt_default = """Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

sys_prompt = input("Enter system prompt: ") or sys_prompt_default
user_prompt = input("Enter user prompt: ") or user_prompt_default

In [24]:
def display_response(model_name, response, stream):
    """Display the response from the model."""
    if stream:
        try:
            res_stream = ""
            display_id = display(Markdown(f"*{model_name} is typing...*"), display_id=True).display_id

            for chunk in response:
                delta = chunk.choices[0].delta.content or ""
                res_stream += delta
                update_display(Markdown(f"*{model_name} is typing...*\n\n{res_stream}"), display_id=display_id)

            update_display(Markdown(f"*{model_name}'s answer:*\n\n{res_stream}"), display_id=display_id)
        except Exception as e:
            update_display(Markdown(f"*Error during streaming:*\n{e}"), display_id=display_id)
    else:
        try:
            display(Markdown(f"*{model_name}'s answer:*\n\n{response.choices[0].message.content}"))
        except Exception as e:
            print(f"Error when trying to get a response from the model:")
            print(f"model name: {model_name}\n{e}")


def ask_model_and_display(client, model_name, sys_prompt, user_prompt, stream=False):
    """Ask the model and display the response, either streamed or not."""
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=stream
        )
    except Exception as e:
        print(f"Error when trying to get a response from the model:")
        print(f"model name: {model_name}\n{e}")
    else:
        display_response(model_name, response, stream)

In [25]:
# Get gpt-4o-mini to answer, with streaming
ask_model_and_display(openai, MODEL_GPT, sys_prompt, user_prompt, stream=True)

Error when trying to get a response from the model:
model name: gpt-4o-mini
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [26]:
# Llama 3.2 with streaming
ask_model_and_display(ollama, MODEL_LLAMA, sys_prompt, user_prompt, stream=True)

*llama3.2's answer:*

**Delegation and Generator Expression**

The provided code is written in Python and utilizes several features that make it concise and efficient.

Let's break down the code:

```python
yield from {book.get("author") for book in books if book.get("author")
```

Here's what each part does:

1. **Generator expression**: `({book.get("author")})` creates a generator expression, which is similar to a list comprehension but returns an iterable instead of a single value.
2. **List comprehension context**: The entire code block is wrapped in the context of a list comprehension `( ...)`.
3. **Condition and iteration**: The generator expression iterates over `books` using `for book in books`. It filters these books based on whether they have an `"author"` key present (`if book.get("author")`).
4. **Yield from**: This keyword is used with yield to allow the execution of another iterable (in this case, a generator expression) and combine its results.

**What does it do?**

In essence, these lines delegate the processing of books' authors to an anonymous inner function (the list comprehension), which returns an iterator over their authors. The `yield from` statement then forwards control to this inner function's first yielded value (`book.get("author")`). As long as there are more values in both contexts, it produces and yields new items one at a time.

**Why use yield from?**

The primary purpose of `yield from` is to make generator expressions more readable by allowing them to be the target of another `yield` statement instead of returning a value themselves. This makes the intent clearer for other developers or for documentation purposes.

In this specific case, using `yield from` simplifies code readability compared to an iterator-based approach, especially when dealing with complex data structures:

```python
def authors():
    for book in books if book.get("author"):
        yield book.get("author")

for author in authors():
    # process each author
```

By delegating the processing of the `books` list to a separate generator expression (`authors()`), and using `yield from`, we can make the code easier to understand without sacrificing performance.

In [27]:
# Llama 3.2 without streaming
ask_model_and_display(ollama, MODEL_LLAMA, sys_prompt, user_prompt, stream=False)

*llama3.2's answer:*

**Generator Expression Explanation**

This line of code uses Python's generator expression syntax to extract author names from a list of book dictionaries.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Let's break it down:

1. **`yield from`**: This keyword is used in Python 3.x to create a **sub-generator** or a sub-iterator. It delegates the iteration process to another iterator.
2. **Dictionary comprehension ( `{...}` )**: Also known as dict comprehensions, they are applied here to filter the books that have an "author" key present and extract their values (`book.get("author")`).
3. **List method call (`for book in books`)**: This is a traditional loop over the `books` list (assumed to be defined elsewhere).

Here's what happens in detail:

1. The code iterates over each book dictionary `book` in the `books` list.
2. For each book, it checks if the "author" key is present (`if book.get("author")`). If not, the `book` dictionary is skipped (because of the `.get()` method, which returns `None` if the key doesn't exist).
3. If the "author" key is present, the author's name (`book.get("author")`) is extracted and made available to the next iteration.
4. The `yield from` statement now comes into play. It creates a new generator that will yield the extracted author names one by one, instead of yielding the entire dictionary.

**Key benefits**

Using `yield from` simplifies the code and makes it more readable. Without `yield from`, you would have to use multiple iterations (loop, then `iter()`, or `next()`). This implementation is more efficient, as it avoids unnecessary objects and overhead.

By using a generator expression with `yield from`, you can avoid mutating state between iterations (because it skips over books without an "author" key), reducing Python's garbage collection effort.